# Welcome to Full Stack Machine Learning's Week 4 Project!

In the final week, you will return to the workflow you built last week on the [taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). 

## Task 1: Deploy the champion
Use what you have learned in the last two weeks to make necessary modifications and to deploy your latest version of the `TaxiFarePrediction` flow to Argo. Use `--branch champion` to denote this deployment as the champion model.

In [17]:
! python ../flows/cloud/event_triggered_linear_regression.py --environment=conda --production --branch champion --with retry argo-workflows create 

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxi_fare_prediction, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taxifareprediction.prod.champion.taxifareprediction to Argo Workflows...

The namespace of this production flow is
    production:mfprj-ovzw7jjg7psagpyw-0-pcke
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-ovzw7jjg7psagpyw-0-pcke")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-ovzw7jjg7psagpyw-0-pcke
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping virtual environment(s) ...
Virtual environment(s) bootstrapped!
Workflow taxifareprediction.prod.champion.taxifareprediction fo

In [18]:
! python ../flows/cloud/event_triggered_linear_regression.py --environment=conda --production --branch champion --with retry argo-workflows trigger

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxi_fare_prediction, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow taxifareprediction.prod.champion.taxifareprediction triggered on Argo Workflows (run-id argo-taxifareprediction.prod.champion.taxifareprediction-z8qp7).
See the run in the UI at https://ui-pw-1910845203.outerbounds.dev/TaxiFarePrediction/argo-taxifareprediction.prod.champion.taxifareprediction-z8qp7


In [23]:
champ_namespace = 'production:mfprj-ovzw7jjg7psagpyw-0-pcke'

## Task 2: Build the challenger
Develop a second model, by using the same `TaxiFarePrediction` architecture. Then, deploy the flow to Argo as the `--branch challenger`. 
<br>
<br>
Hint: Modify the `linear_model` step. 
<br>
Bonus: Write a paragraph summary of how you developed the second model and tested it before deploying the challenger flow. Let us know in Slack what you found challenging about the task? 

In [39]:
! python ../flows/cloud/event_triggered_xgboost.py --environment=conda --production --branch challenger --with retry argo-workflows create
! python ../flows/cloud/event_triggered_xgboost.py --environment=conda --production --branch challenger --with retry argo-workflows trigger

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxi_fare_prediction, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taxifareprediction.prod.challenger.taxifareprediction to Argo Workflows...

The namespace of this production flow is
    production:mfprj-6iffsxtybx6fkjku-0-crtx
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-6iffsxtybx6fkjku-0-crtx")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-6iffsxtybx6fkjku-0-crtx
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping virtual environment(s) ...
Virtual environment(s) bootstrapped!
Workflow taxifareprediction.prod.challenger.taxifarepredict

In [40]:
challenger_namespace = 'production:mfprj-6iffsxtybx6fkjku-0-crtx'

## Task 3: Analyze the results
Return to this notebook, and read in the results of the challenger and champion flow using the Metaflow Client API.
<br><br>

#### Questions
- Does your model perform better on the metrics you selected? 
- Think about your day job, how would you go about assessing whether to roll forward the production "champion" to your new model? 
    - What gives you confidence one model is better than another?
    - What kinds of information do you need to monitor to get buy-in from stakeholders that model A is preferable to model B?  

In [41]:
namespace('user:sandbox')
Metaflow().flows

[Flow('HelloFlow'),
 Flow('TFlow1'),
 Flow('BaselineNLPFlow'),
 Flow('GoodFirstNLPFlow'),
 Flow('TFlow5'),
 Flow('BaselineChallenge'),
 Flow('NLPFlow'),
 Flow('RetryFlow'),
 Flow('TimeoutFlow'),
 Flow('Branch_Flow_Cloud'),
 Flow('DivideByZeroFlow'),
 Flow('TaxiFarePrediction'),
 Flow('CatchDivideByZeroFlow'),
 Flow('TitanicSurvivalPredictor')]

In [43]:
from metaflow import Flow, namespace

best_score = -1; winner = None; winner_namespace = None
model_space = {'champion':champ_namespace, 'challenger':challenger_namespace}
for i in model_space:
    namespace(model_space[i])
    run = Flow('TaxiFarePrediction').latest_successful_run
    r_square_score = run.data.scores.mean()
    if r_square_score > best_score:
        best_score = r_square_score
        winner = i
        winner_namespace = model_space[i]

In [45]:
print("The winner is model " + str(i) +" and the winner's score is " + str(best_score))

The winner is model challenger and the winner's score is 0.9046101615141199


## CONGRATULATIONS! 🎉✨🍾
If you made it this far, you have completed the Full Stack Machine Learning Corise course. 
We are so glad that you chose to learn with us, and hope to see you again in future courses. Stay tuned for more content and come join us in [Slack](http://slack.outerbounds.co/) to keep learning about Metaflow!